# UDX Worker Interface

This notebook serves as the interface for the UDX worker. It provides the necessary setup and function registrations. It also includes a basic local testing of operational tasks running a UDX worker based Docker container.

In [ ]:
# Import the necessary libraries
import docker
import os

# Initialize the Docker client
client = docker.from_env()

# Your JavaScript code
js_code = """
/**
 * 
 * Mock service that logs a message every 5 seconds.
 * 
 */

function sleep(seconds) {
    const milliseconds = seconds * 1000;
    const start = Date.now();
    while (true) {
      const current = Date.now();
      if (current - start >= milliseconds) {
        break;
      }
    }
}

while (true) {
    console.log(`[${new Date().toLocaleTimeString()}] Service is up and running!`);
    sleep(5);
}
"""

# Write the JavaScript code to a file
with open('script.js', 'w') as file:
    file.write(js_code)

# Pull your Docker image
# Replace 'your-docker-image' with the name of your Docker image
image = client.images.pull('your-docker-image')

# Run a Docker container from your image
# In this case, the JavaScript file is in the same directory as this Python script
container = client.containers.run('your-docker-image', command=['node', './script.js'], volumes={os.getcwd(): {'bind': '/usr/src/app', 'mode': 'rw'}}, detach=True)

# Fetch the logs of the container
logs = container.logs()

# Print the logs
print(logs)